In [ ]:
!pip install crcmod

In [ ]:
import serial
import crcmod
import struct

# COM port configuration
RelayCOMPort = 'COM4'

In [ ]:
# Open serial port at com 8
def openSerial(COM: str):
    serialPort = serial.Serial(COM, 9600, timeout=1)
    return serialPort

RelayCOM = openSerial(RelayCOMPort)

In [ ]:
# close serial port
def closeSerial(serialPort: str):
    serialPort.close()

closeSerial(RelayCOMPort)

In [ ]:
# seiral port write in hex
def writeHexSerial(serialPort: str, command: str):
    hex_command = bytes.fromhex(command)
    serialPort.write(hex_command)
    response = serialPort.read(serialPort.in_waiting or 1)
    return response

# all relay on command
allRelayOn = '01 06 00 34 00 01 09 C4'
allRelayOff = '01 06 00 34 00 00 C8 04'

PWM1 = '01 06 00 77 01 F4 39 C7'

In [20]:
import serial
import time

def writeHexSerial(port: str, baudrate: int, command: str):
    # Convert the command string to bytes
    hex_command = bytes.fromhex(command)
    
    # Open the serial port
    with serial.Serial(port, baudrate, timeout=1) as ser:
        # Flush any existing input/output before sending the new command
        ser.reset_input_buffer()
        ser.reset_output_buffer()
        
        # Write the command
        ser.write(hex_command)
        
        # Short delay to ensure data is transmitted and response is received
        time.sleep(2)
        
        # Read the response
        response = ser.read(ser.in_waiting)
 
        
    # Convert the response to hexadecimal for easy comparison and return it
    return response

# Example usage
port = "COM4"  # Adjust this to your serial port name (e.g., COM3 on Windows)
baudrate = 9600  # Adjust based on your device's baud rate
command = "0106007701F439C7"

response = writeHexSerial(port, baudrate, command)
print("Response:", response)


Response: b'?\xc7\xc1\xe5\xc0\xe2\xe4?\xc7\xe1P\xc0\xe2\xe4?\xc7\xe1P0\xa9\xc8?\xc7\xe1P\xc0\xe2\xe4'


In [ ]:
def writeByteSerial(serialPort: str, command: bytes):
    serialPort.write(command)
    response = serialPort.read(serialPort.in_waiting or 1)
    return response

In [ ]:
response = writeHexSerial(RelayCOM, PWM1)
print(response)

In [ ]:
response = writeHexSerial(RelayCOM, allRelayOff)
print(response) 

In [ ]:
# Use library to calculate CRC-16 Modbus for the last two byte of a command
command_1 = '01 06 00 34 00 01'

# Convert the command to bytes
command_bytes = bytes.fromhex(command_1.replace(" ", ""))

# Define the CRC-16 Modbus function
crc16 = crcmod.mkCrcFun(0x18005, rev=True, initCrc=0xFFFF, xorOut=0x0000)

# Calculate the CRC
crc_result = crc16(command_bytes)

# reverse the bytes order
crc_result = struct.pack("<H", crc_result)

# command_1b = binascii.hexlify(crc_result)

# Print the result in hexadecimal
print(crc_result.hex())

In [ ]:
# practice to use string based command to obtain crc
# combine the byte form of command with crc (byte form)
# send out the command

command_1_hex = bytes.fromhex(command_1)
print(command_1_hex)
full_command_1 = command_1_hex + crc_result
print(full_command_1)
response = writeHexSerial(RelayCOM, full_command_1.hex())

In [ ]:
# practice with decimal, hex and bytes
pos_1 = 1
print(pos_1)
# bytes of pos_1
pos_1_bytes = pos_1.to_bytes(1, byteorder='big')
print(pos_1_bytes)
# note that the result is in hex \x01 instead of 1
# bytes and binary are different